In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# basic import
import os
import os.path as op
import ast
import sys
sys.path.insert(0, op.dirname(os.getcwd()))

# python libs
import pickle
import numpy as np
import xarray as xr
from datetime import datetime

# tk libs
from lib.objs.tkpaths import Site
from lib.objs.predictor import Predictor
from lib.io.matlab import ReadGowMat
from lib.KMA import Persistences
#from lib.extremes import FitGEV_KMA_Frechet as FitGEV
#from lib.extremes import SampleGEV_KMA_Smooth as SampleGEV
#from lib.extremes import ChromosomesProbabilities_KMA as ChromProbs
#from lib.extremes import Correlation_Smooth_Partitions as CorrSP
from lib.extremes import Climate_Emulator as CE


# --------------------------------------
# Site paths and parameters
site = Site('KWAJALEIN')

DB = site.pc.DB                        # common database
ST = site.pc.site                      # site database
PR = site.params                       # site parameters

# input files
p_WVS_MaxStorm = ST.EXTREMES.waves_maxstorm     # waves families at max storm TWL
p_KMA_MaxStorm = ST.EXTREMES.kma_maxstorm       # KMA at max storm TWL

p_dwt_sim = ST.ESTELA.sim_dwt                   # daily weather types simulated with ALR

p_sim_r2_params = ST.TCs.sim_r2_params          # TCs parameters (copula generated) 
p_sim_r2_RBF_output = ST.TCs.sim_r2_rbf_output  # TCs numerical_IH-RBFs_interpolation output

p_mu_tau = ST.ESTELA.hydrographs                # intradaily mu tau hydrographs

p_probs_synth = ST.TCs.probs_synth              # synthetic TCs probabilities

p_wvs_fams_TCs_c = ST.WAVES.families_tcs_categ  # waves families at TCs time window by categories (folder)



# output files



In [6]:
# --------------------------------------
# Load data

# TODO: asi se cargan los datos generados por python teslakit. Pero algunos no estan coordinados con el caso matlab
# - El predictor ESTELA usa KMA, es aleatorio.
# - MU TAU dependen del KMA, no coincidiran
# - los params de mi generacion-seleccion TCs son aleatorios, no coinciden con los simulados en IH hace tiempo
# - los DWT tambien dependen de generacion aleatoria tanto en MJO_SIM, como en PCS123_SIM, como en el propio ALR 

# - comparar con matlab los wt37-42: resolucion datos, gestion nans vs 0s, testear que salga correcto coger los datos
# - repasar las probabilidades TCs synth, he visto que la matriz de probs_change no sale completa


# ESTELA predictor KMA at max. storms
xds_WVS_MaxStorm = xr.open_dataset(p_WVS_MaxStorm)

# waves families at max. storms
xds_WVS_MaxStorm = xr.open_dataset(p_WVS_MaxStorm)

# DWTs (Daily Weather Types simulated bmus using ALR)
xds_DWT = xr.open_dataset(p_dwt_sim)
# TODO: pasar de 1 dato por dia a 1 dato por tormenta (como en maxStorm)

# TCs simulated with numerical and RBFs (parameters and num/RBF output)
xds_TCs_params = xr.open_dataset(p_sim_r2_params)
xds_TCs_RBFs = xr.open_dataset(p_sim_r2_RBF_output)

xds_TCs = xr.merge([
    xds_TCs_params.pressure_min,
    xds_TCs_RBFs.hs,
    xds_TCs_RBFs.tp,
    xds_TCs_RBFs.dir,
    xds_TCs_RBFs.mu,
    xds_TCs_RBFs.ss,
])

# MU - TAU intradaily hidrographs for each WWT
# TODO tener MU_TAU ya preparado en un .npy
d_bins_mutau = pickle.load(open(p_mu_tau, 'rb')) 
MU_WT = []
TAU_WT = []
for k in sorted(d_bins_mutau.keys()):
    hg = d_bins_mutau[k]
    MU_WT.append(np.array(hg.MU))
    TAU_WT.append(np.array(hg.TAU))
MU_WT = np.array(MU_WT)
TAU_WT = np.array(TAU_WT)

# Synth. TCs probabilitie changues
xds_probs_TCs = xr.open_dataset(p_probs_synth)
pchange_TCs = xds_probs_TCs['category_change_count'].values[:]

# Load TCs-window waves-families data by category
d_WT_TCs_wvs = {}
for k in range(6):
    p_s = op.join(p_wvs_fams_TCs_c, 'waves_fams_cat{0}.nc'.format(k))
    d_WT_TCs_wvs['{0}'.format(k)] = xr.open_dataset(p_s)


In [ ]:
# --------------------------------------
# Simulate with Climate Emulator

CE()
